In [22]:
import hist
import numpy as np
from plot_utils import adjust_plot
import matplotlib.pyplot as plt
import pickle
from utils_v1 import Histfit, computeJER
from scipy.optimize import curve_fit
import matplotlib.gridspec as gridspec

In [23]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import mplhep as hep
hep.style.use("CMS")

In [25]:
year = 2018
era = "2018"

if era == "2016":
    jerfile = "Summer20UL16_JRV3_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2016.txt'
    year =2016
    
if era == '2016APV':
    jerfile = "Summer20UL16APV_JRV3_MC_PtResolution_AK4PFchs.txt"    
    filename = 'samples/flatPU_JMENano_2016APV.txt'
    year = 2016
if era == '2017':
    jerfile = "Summer20UL17_JRV1_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2017.txt'
    year = 2017
    more = 'JME'
if era == '2018':
    jerfile = "Summer20UL18_JRV1_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2018.txt'
    year = 2018
    more = 'JME'  #change more according to filename

In [26]:
# with open("QCD_pt_response_"+era+more+".pkl", "rb") as f:
#     output = pickle.load( f )

with open("QCDresponse_2018JME.pkl", "rb") as f:
    output = pickle.load( f )

In [27]:
hist_0 = output["pt_reco_over_gen"]
hist_0.axes

(StrCategory(['2018'], growth=True, name='dataset', label='Primary dataset'),
 Variable(array([   10.,    11.,    12.,    13.,    14.,    15.,    17.,    20.,
           23.,    27.,    30.,    35.,    40.,    45.,    57.,    72.,
           90.,   120.,   150.,   200.,   300.,   400.,   550.,   750.,
         1000.,  1500.,  2000.,  2500.,  3000.,  3500.,  4000.,  5000.,
        10000.]), name='pt', label='$p_{T}$ [GeV]'),
 Regular(300, 0, 2, name='frac', label='Fraction'),
 Variable([0, 0.5, 0.8, 1.1, 1.3, 1.7, 1.9, 2.1, 2.3, 2.5, 2.8, 3, 3.2, 4.7], name='eta', label='$\\eta$'),
 Variable([0, 7.47, 13.49, 19.52, 25.54, 31.57, 37.59, 90], name='rho', label='$\\rho$'),
 Variable([0, 10, 20, 31, 43, 56], name='pileup', label='$\\mu$'))

In [28]:
datasets = hist_0.axes[0].centers
pt_bins = hist_0.axes[1].edges
frac_bins = hist_0.axes[2].edges
eta_bins = hist_0.axes[3].edges
rho_bins = hist_0.axes[4].edges
pileup_bins = hist_0.axes[5].edges


pt_values = hist_0.axes[1].centers
frac_values = hist_0.axes[2].centers
eta_values = hist_0.axes[3].centers
rho_values = hist_0.axes[4].centers
pileup_values = hist_0.axes[5].centers

pt_widths = hist_0.axes[1].widths
frac_widths = hist_0.axes[2].widths
eta_widths = hist_0.axes[3].widths
rho_widths = hist_0.axes[4].widths
pileup_widths = hist_0.axes[5].widths

In [29]:
def jerfunc(x, p0, p1, p2, p3):
    return np.sqrt(p0*np.abs(p0)/(x*x)+p1*p1*np.power(x,p3) + p2*p2)

In [30]:
import pandas as pd
#eras = ['2017', '2018']
eras = ['2018']
df_fit = pd.DataFrame({"pileup_low":[], "pileup_high":[], "eta_low":[], "eta_high":[], "par0":[],"par0_unc":[], "par1":[],"par1_unc":[],
                       "par2":[],"par2_unc":[], "par3":[],"par3_unc":[] , "era":[]})
n_rho_min = 0
n_rho_max = 6
n_eta_min = 0
n_eta_max = 15
n_rho = n_rho_max - n_rho_min
n_eta = n_eta_max - n_eta_min
for era in eras:

    if era == "2016":
        jerfile = "Summer20UL16_JRV3_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2016.txt'
        year =2016
        more = "_premix"
    if era == '2016APV':
        jerfile = "Summer20UL16APV_JRV3_MC_PtResolution_AK4PFchs.txt"    
        filename = 'samples/flatPU_JMENano_2016APV.txt'
        year = 2016
        more = "_premix"
    if era == '2017':
        jerfile = "Summer20UL17_JRV1_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2017.txt'
        year = 2017
        more = "JME_all_jets"
    if era == '2018':
        jerfile = "Summer20UL18_JRV1_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2018.txt'
        year = 2018
        more = "JME"
    import itertools
    # with open("QCD_pt_response_"+era+more+".pkl", "rb") as f:
    #     output = pickle.load( f )
    with open("QCDresponse_2018JME.pkl", "rb") as f:
        output = pickle.load( f )
    hist_0 = output["pt_reco_over_gen"]
    
    for i_rho in range(n_rho_min, n_rho_max):
        for i_eta in range(n_eta_min, n_eta_max):
            hist_rho_pt_frac = hist_0.project("eta","pileup","pt","frac").to_numpy()[0][i_eta]
            
            
            markers = itertools.cycle(['o', 's', 'v', '^', 'D'])
            hist_pt_frac = hist_rho_pt_frac[i_rho]
            histfit = Histfit(hist_pt_frac, frac_values, pt_values)
            histfit.store_parameters()

            sigma_list = histfit.parameters["sigma"]
            sigma_error_list  = np.array(histfit.parameters["sigmaErr"])
            

            sel = (histfit.parameters["sigma"] != None ) & (pt_values>=30) 

            popt,pcov = curve_fit(jerfunc,  np.array(pt_values[sel], dtype = 'float64'),  np.array(sigma_list[sel], dtype = 'float64') , 
                                  sigma = np.array(sigma_error_list[sel],dtype = 'float64') , bounds =([-10,-5,-3,-5],[10,2,2,5]) )    

            df_fit2 = pd.DataFrame({"pileup_low":[pileup_bins[i_rho]], "pileup_high":[pileup_bins[i_rho+1]], "eta_low":[eta_bins[i_eta]], "eta_high":[eta_bins[i_eta+1]], 
                                    "par0":[popt[0]],"par0_unc":[np.sqrt(pcov[0][0])], "par1":[popt[1]],"par1_unc":[np.sqrt(pcov[1][1])],
                                    "par2":[popt[2]],"par2_unc":[np.sqrt(pcov[2][2])], "par3":[popt[3]],"par3_unc":[np.sqrt(pcov[3][3])] , "era":[era]})
            df_fit = pd.concat([df_fit,df_fit2])
            


ValueError: `ydata` must not be empty!

In [32]:
histfit.parameters

{'mean': array([None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None],
       dtype=object),
 'sigma': array([None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None],
       dtype=object),
 'const': array([None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None],
       dtype=object),
 'sigmaErr': array([None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None],
       dtype=object)}